# Ensemble Tweets - Level 2

## Changes:

# Calculated the coefficients of the level-2 Logistic Regression model & mapped the coefficients with the models in a dictionary.

In [1]:
import pandas as pd

In [2]:
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')


In [3]:
x2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_probas.csv') # Train : Set-2 Predict_probas
x3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_probas.csv')# Val : Set-3 Predict_probas
test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_probas.csv')

Bert2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_Bert_proba.csv')
Bert3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_Bert_proba.csv')
Bert_test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_data_Bert_proba.csv')

Roberta2 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set2_Roberta_proba.csv')
Roberta3 = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/set3_Roberta_proba.csv')
Roberta_test = pd.read_csv('/content/drive/My Drive/Tweets/5. Predict Probas/test_data_Roberta_proba.csv')


In [4]:
Bert_test[:2]

,Bert_1,Bert_0
0,0.954075,0.045925
1,0.965994,0.034006


In [5]:
#Adding Bert Probas to x2, x3, test

x2['Bert_1'] = Bert2['Bert_1']
x2['Bert_0'] = Bert2['Bert_0']

x3['Bert_1'] = Bert3['Bert_1']
x3['Bert_0'] = Bert3['Bert_0']

test['Bert_1'] = Bert_test['Bert_1']
test['Bert_0'] = Bert_test['Bert_0']


# Adding Robert Probas to x2, x3, test

x2['Roberta_1'] = Roberta2['Roberta_1']
x2['Roberta_0'] = Roberta2['Roberta_0']

x3['Roberta_1'] = Roberta3['Roberta_1']
x3['Roberta_0'] = Roberta3['Roberta_0']

test['Roberta_1'] = Roberta_test['Roberta_1']
test['Roberta_0'] = Roberta_test['Roberta_0']

In [24]:
x2.shape

(1903, 16)

In [6]:
x2[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.3631,0.6369,0.418081,0.581919,0.578602,0.421398,0.536737,0.463263,0.54,0.46,0.456238,0.543762,0.388212,0.611788,0.188238,0.811762


In [7]:
x3[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.174052,0.825948,0.907352,0.092648,0.089012,0.910988,0.227634,0.772366,0.06,0.94,0.456238,0.543762,0.057976,0.942024,0.073738,0.926262


In [8]:
test[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.276864,0.723136,0.430683,0.569317,0.649411,0.350589,0.191533,0.808467,0.4,0.6,0.522109,0.477891,0.954075,0.045925,0.986153,0.013847


In [9]:
y2 = set2.target
y3 = set3.target

In [10]:
y3.value_counts()

0    1073
1     831
Name: target, dtype: int64

In [11]:
x2['Bert_1'][:5].values.tolist()

[0.3882122039794922,
 0.9852584600448608,
 0.4531148076057434,
 0.9874793887138368,
 0.1129818707704544]

In [12]:
def deci(r):
  return '%.4f'%r

In [13]:
x2.columns, x3.columns, test.columns

(Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'),
 Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'),
 Index(['gb1', 'gb0', 'lr1', 'lr0', 'svc1', 'svc0', 'bayes1', 'bayes0', 'tree1',
        'tree0', 'cat1', 'cat0', 'Bert_1', 'Bert_0', 'Roberta_1', 'Roberta_0'],
       dtype='object'))

In [14]:
cols = x2.columns.tolist()

In [15]:
for i in cols:
  x2[i] = x2[i].apply(deci)
  x2[i] = x2[i].astype(float)

  x3[i] = x3[i].apply(deci)
  x3[i] = x3[i].astype(float)

  test[i] = test[i].apply(deci)
  test[i] = test[i].astype(float)

In [16]:
test[:1]

,gb1,gb0,lr1,lr0,svc1,svc0,bayes1,bayes0,tree1,tree0,cat1,cat0,Bert_1,Bert_0,Roberta_1,Roberta_0
0,0.2769,0.7231,0.4307,0.5693,0.6494,0.3506,0.1915,0.8085,0.4,0.6,0.5221,0.4779,0.9541,0.0459,0.9862,0.0138


In [17]:
!pip install -q category_encoders
import category_encoders as ce

     |████████████████████████████████| 81kB 2.1MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear',penalty='l1')

In [20]:
model = Pipeline([('enc', ce.LeaveOneOutEncoder()),
                  ('clf', lr ) ])
model.fit(x2,y2)

pred2 = model.predict(x2)
print('Rounded F1 SCORE on set-2 : %.2f'%f1_score(y2, pred2, average='binary'))
print('Exact F1 SCORE on set-2 = ',f1_score(y2, pred2, average='binary'))
print('*'*55)

pred = model.predict(x3)
print('Rounded F1 SCORE on set-3 : %.2f'%f1_score(y3, pred, average='binary'))
print('Exact F1 SCORE on set-3 = ',f1_score(y3, pred, average='binary'))
print('*'*55)

tn, fp, fn, tp = confusion_matrix(y3, pred).ravel()
print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
print('*'*55)

print(classification_report(y3, pred))
print('*'*55)

pred_proba = model.predict_proba(x3)
test_proba = model.predict_proba(test)

Rounded F1 SCORE on set-2 : 0.81
Exact F1 SCORE on set-2 =  0.8095238095238096
*******************************************************
Rounded F1 SCORE on set-3 : 0.82
Exact F1 SCORE on set-3 =  0.8185185185185184
*******************************************************

TP : 663 	FN : 168 
TN : 947 	FP : 126 


*******************************************************
              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1073
           1       0.84      0.80      0.82       831

    accuracy                           0.85      1904
   macro avg       0.84      0.84      0.84      1904
weighted avg       0.85      0.85      0.85      1904

*******************************************************


# Logistic Regression Coefficients

In [43]:
coefficient_arr = model.named_steps['clf'].coef_

In [42]:
coefficients = list(coefficient_arr.flat)

In [44]:
all_models = x2.columns.tolist()

In [45]:
models_with_coeffcients = dict(zip(all_models, coefficients))

In [46]:
models_with_coeffcients

{'Bert_0': -1.3342321866964524,
 'Bert_1': 0.4596411213209439,
 'Roberta_0': -1.271919202706957,
 'Roberta_1': 1.8775771671334571,
 'bayes0': 0.0,
 'bayes1': 0.0,
 'cat0': 0.0,
 'cat1': 0.0,
 'gb0': -0.4199184750797515,
 'gb1': 0.775481767246767,
 'lr0': 0.0,
 'lr1': -0.00026902030352710885,
 'svc0': -0.1319465179810268,
 'svc1': 0.0,
 'tree0': 0.0,
 'tree1': 0.0}

In [47]:
model.named_steps['clf'].coef_

array([[ 7.75481767e-01, -4.19918475e-01, -2.69020304e-04,
         0.00000000e+00,  0.00000000e+00, -1.31946518e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.59641121e-01, -1.33423219e+00,  1.87757717e+00,
        -1.27191920e+00]])

In [ ]:
y3[10:20].tolist()

In [ ]:
pred[10:20] #0.3

In [ ]:
pred_proba[10:20] # 0,1 # 1st col: non-disaster proba; 2nd col: Disaster proba

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(model.predict_proba(test)[:,0])

In [ ]:
plt.hist(model.predict_proba(test)[:,1])

In [ ]:
pred


# ADJUSTING THE THRESHOLD

In [ ]:
from sklearn.preprocessing import binarize

In [ ]:

def all_metrics(n):
  pred = binarize(pred_proba,n)
  predictions = pred[:,1]
  predictions = predictions.astype(int)
  print('F1 SCORE rounded : %.2f'%f1_score(y3, predictions, average='binary'))
  print('\n\nF1 SCORE = ' ,f1_score(y3, predictions, average='binary'))
  tn, fp, fn, tp = confusion_matrix(y3, predictions).ravel()
  print(f'\nTP : {tp} \tFN : {fn} \nTN : {tn} \tFP : {fp} \n\n')
  print('*'*55)

  print(classification_report(y3, predictions))
  print('*'*55)

# DEFAULT THRESHOLD : 0.5

In [ ]:
all_metrics(0.5) 

# FINDING THE THRESHOLD WITH THE BEST F1 SCORE


In [ ]:
def threshold_selection(n):
  pred = binarize(pred_proba,n)
  predictions = pred[:,1]
  predictions = predictions.astype(int)
  
  return f1_score(y3, predictions, average='binary')
  


In [ ]:
q = threshold_selection(0.5)
q

In [ ]:
import numpy as np

# Loop through 100 numbers from 0.1 to 1.0 to find the best threshold

In [ ]:
f1 = 0.8185185185185184
t = 0.5
# add for loop 0.1-0.99 ; find the value of n for high overall f1 score
for n in np.arange(0.1,1.0,0.01):
  if threshold_selection(n) > f1:
    f1 = threshold_selection(n)
    t = n

print(f1)
print(t)

In [ ]:
t1 = threshold_selection(0.5)
t2 = threshold_selection(0.51)

print(t1)
print(t2)


# NEW THRESHOLD


In [ ]:
all_metrics(0.51)

# CHANGING THE THRESHOLD TO 0.52

In [ ]:

test_proba

In [ ]:
test_proba_new = binarize(test_proba,0.51)

In [ ]:
test_proba_new[:20]

In [ ]:

predictions = test_proba_new[:,1]
predictions[:10]

In [ ]:
predictions = predictions.astype(int)
predictions[:10]

In [ ]:
predictions = predictions.tolist()
len(predictions)

# Load Test Data

In [ ]:
submission = pd.read_csv('/content/drive/My Drive/Tweets/2. Input/tweets_sample_submission.csv')

In [ ]:
output = pd.DataFrame({'id': submission.id, 'target': predictions})

In [ ]:
output[:5]

In [ ]:
output.to_csv('submission.csv', index=False) 
!cp submission.csv "/content/drive/My Drive/Tweets/7. Output" 